In [0]:
import pandas as pd
import numpy as np
from numpy import loadtxt
import matplotlib.pyplot as plt
import cv2
import sys, os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, LeakyReLU
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.layers import Dropout
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings("ignore")

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
data=pd.read_csv('gdrive/My Drive/Datasets/four_emotions.csv')

In [0]:
data=shuffle(data)
data.head()

,emotion,pixels,anger,happiness,neutral,sadness,nf
18621,1,241 112 28 44 51 80 101 106 129 162 181 190 19...,7,3,0,0,0
27714,2,49 35 32 29 24 26 29 34 35 34 36 38 32 30 28 3...,0,9,1,0,0
23716,3,28 34 42 47 46 39 27 29 21 27 41 75 99 117 136...,0,10,0,0,0
8169,2,154 155 140 152 179 181 155 163 200 198 193 19...,0,9,1,0,0
33804,2,84 95 108 102 100 136 150 139 103 108 120 125 ...,0,1,4,2,3


In [0]:
image=data['pixels'].tolist()

In [0]:
data = data.drop(['pixels', 'emotion', 'nf'], axis=1)
y = data
y=np.array(y)
y.shape

(35887, 4)

In [0]:
X = []
for i in range(len(image)):
    d=[int(x) for x in image[i].split(' ')]
    d = np.asarray(d).reshape(48, 48)
    X.append(d.astype('float32'))

In [0]:
del(data)
del(image)
del(d)
x_new=X

In [0]:
#x_new = []
#for i in range(len(X)):
#    d = cv2.resize(X[i], (200, 200), fx=0.5, fy=0.5, interpolation = cv2.INTER_CUBIC)
#    d = np.asarray(d)
#    x_new.append(d.astype('float32'))
#    del(d)

In [0]:
del(X)

In [0]:
epochs=100
batch_size = 64
num_features = 64
output_size = 4
alpha=0.05
reg=0.001
width = 200
height = 200

In [0]:
x_new = np.asarray(x_new)
x_new=x_new.reshape(-1,48,48,1)
x_new.shape

(35887, 48, 48, 1)

In [0]:
x_new -= np.mean(x_new, axis=0)
x_new /= np.std(x_new, axis=0)

In [0]:
#np.save('Ofer_x_new', x_new)
#np.save('Ofer_y', y)
#!mv fer_x_new.npy 'gdrive/My Drive/Datasets/FER2013/Ofer_x_new.npy'
#!mv fer_y.npy 'gdrive/My Drive/Datasets/FER2013/Ofer_y.npy'
#x_new=np.load('gdrive/My Drive/Datasets/FER2013/fer_x_new.npy')
#y=np.load('gdrive/My Drive/Datasets/FER2013/fer_y.npy')

In [0]:
#from google.colab import files
#files.download('fer_x_new.npy')
#files.download('fer_y.npy')

In [0]:
def augment_data(dataset, dataset_labels, augementation_factor=1, use_random_rotation=True, use_random_shear=True, use_random_shift=True, use_random_zoom=True):
    augmented_image = []
    augmented_image_labels = []

    for num in range (0, dataset.shape[0]):

        for i in range(0, augementation_factor):
            # original image:
            augmented_image.append(dataset[num])
            augmented_image_labels.append(dataset_labels[num])

            if use_random_rotation:
                augmented_image.append(tf.contrib.keras.preprocessing.image.random_rotation(dataset[num], 20, row_axis=0, col_axis=1, channel_axis=2))
                augmented_image_labels.append(dataset_labels[num])

            if use_random_shear:
                augmented_image.append(tf.contrib.keras.preprocessing.image.random_shear(dataset[num], 0.2, row_axis=0, col_axis=1, channel_axis=2))
                augmented_image_labels.append(dataset_labels[num])

            if use_random_shift:
                augmented_image.append(tf.contrib.keras.preprocessing.image.random_shift(dataset[num], 0.2, 0.2, row_axis=0, col_axis=1, channel_axis=2))
                augmented_image_labels.append(dataset_labels[num])

            #if use_random_zoom:
             #   augmented_image.append(tf.contrib.keras.preprocessing.image.random_zoom(dataset[num], 0.9, row_axis=0, col_axis=1, channel_axis=2))
              #  augmented_image_labels.append(dataset_labels[num])

    return np.array(augmented_image), np.array(augmented_image_labels)
x_new, y = augment_data(x_new, y)

In [0]:
x_train, x_valid, y_train, y_valid = train_test_split(x_new, y, test_size=0.2, random_state=42)

In [0]:
del(x_new)
del(y)

In [0]:
labels = [ 'Angry', 'Happy', 'Neutral', 'Sad']

In [0]:
def signs_model(width, height, output_size, alpha=0.05):
  
  model = Sequential()

  model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
  model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  model.add(Dropout(0.1))

  model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  model.add(Dropout(0.1))

  model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  model.add(Dropout(0.1))

  model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  model.add(Dropout(0.1))

  model.add(Flatten())

  model.add(Dense(2*2*2*num_features, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(2*2*num_features, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(2*num_features, activation='relu'))
  model.add(Dropout(0.5))

  model.add(Dense(output_size, activation='softmax'))
  return model

In [0]:
model=signs_model(48, 48, output_size, alpha)
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])

In [0]:
#model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=len(x_train)//batch_size, epochs=epochs, 
#          verbose=1, validation_data=(np.array(x_valid), np.array(y_valid)), shuffle=True)#, 
#                    #callbacks=[EarlyStopping(monitor='val_acc', mode='auto', patience=6)])
model.fit(x_train, y_train, batch_size=512, epochs=epochs, 
         verbose=1, validation_data=(x_valid, y_valid), shuffle=True, callbacks=[EarlyStopping(monitor='val_acc', mode='auto', patience=6)])

Train on 114838 samples, validate on 28710 samples
Epoch 1/100
114838/114838 [==============================] - 86s 751us/step - loss: 11.7676 - acc: 0.4993 - val_loss: 9.9308 - val_acc: 0.6015
Epoch 2/100
114838/114838 [==============================] - 82s 718us/step - loss: 8.7406 - acc: 0.6816 - val_loss: 7.9233 - val_acc: 0.7264
Epoch 3/100
114838/114838 [==============================] - 83s 725us/step - loss: 7.5119 - acc: 0.7636 - val_loss: 7.0154 - val_acc: 0.7862
Epoch 4/100
114838/114838 [==============================] - 84s 728us/step - loss: 6.9180 - acc: 0.8030 - val_loss: 6.7127 - val_acc: 0.8097
Epoch 5/100
114838/114838 [==============================] - 84s 731us/step - loss: 6.5395 - acc: 0.8314 - val_loss: 6.5209 - val_acc: 0.8209
Epoch 6/100
114838/114838 [==============================] - 84s 731us/step - loss: 6.2551 - acc: 0.8512 - val_loss: 6.3087 - val_acc: 0.8398
Epoch 7/100
114838/114838 [==============================] - 84s 732us/step - loss: 6.0245 - acc

In [0]:
model.save('model.h5')
!mv model.h5 'gdrive/My Drive/Datasets/Models/fer.h5'

In [0]:
from google.colab import files
files.download('fer.h5')

In [0]:
from PIL import Image
import os, os.path

In [0]:
imgs = []
y = []
path = "gdrive/My Drive/Datasets/JAFF/NEUTRAL"
ext = [".jpg"]
for f in os.listdir(path):
  ext = os.path.splitext(f)[1]
  imgs.append(Image.open(os.path.join(path,f)))
  y.append(1)

In [0]:
path = "gdrive/My Drive/Datasets/JAFF/HAPPY"
ext = [".jpg"]
for f in os.listdir(path):
  ext = os.path.splitext(f)[1]
  imgs.append(Image.open(os.path.join(path,f)))
  y.append(2)

In [0]:
path = "gdrive/My Drive/Datasets/JAFF/SAD"
ext = [".jpg"]
for f in os.listdir(path):
  ext = os.path.splitext(f)[1]
  imgs.append(Image.open(os.path.join(path,f)))
  y.append(3)

In [0]:
path = "gdrive/My Drive/Datasets/JAFF/ANGRY"
ext = [".jpg"]
for f in os.listdir(path):
  ext = os.path.splitext(f)[1]
  imgs.append(Image.open(os.path.join(path,f)))
  y.append(4)

In [0]:
x = []
for i in range(len(imgs)):
    d = np.asarray(imgs[i]).reshape(256, 256)
    d=cv2.resize(d, (200, 200), fx=0.5, fy=0.5, interpolation = cv2.INTER_CUBIC)
    x.append(d.astype('float32'))

In [0]:
x = np.asarray(x)
x = np.expand_dims(x, -1)
x.shape

(152, 200, 200, 1)

In [0]:
y=pd.get_dummies(y)
y.shape

(152, 4)

In [0]:
#np.save('jeff_x', x)
#np.save('jeff_y', y)
#!mv jeff_x.npy 'gdrive/My Drive/Datasets/FER2013/jeff_x.npy'
#!mv jeff_y.npy 'gdrive/My Drive/Datasets/FER2013/jeff_y.npy'
#x=np.load('gdrive/My Drive/Datasets/FER2013/jeff_x.npy')
#y=np.load('gdrive/My Drive/Datasets/FER2013/jeff_y.npy')

In [0]:
model=load_model('gdrive/My Drive/Datasets/FER2013/model.h5')
#model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
#model=signs_model(width, height, output_size, alpha)
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])

In [0]:
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, rotation_range=20, 
                             zoom_range=0.15, rescale=1./255,width_shift_range=0.2, height_shift_range=0.2, 
                             shear_range=0.2, horizontal_flip=True, fill_mode="nearest")
datagen.fit(x)

In [0]:
model.fit_generator(datagen.flow(x, y, batch_size=32), steps_per_epoch=len(x)//8, epochs=epochs, 
          verbose=1, shuffle=True)#, 
#                    #callbacks=[EarlyStopping(monitor='val_acc', mode='auto', patience=6)])
#model.fit(np.array(x), np.array(y), batch_size=None, epochs=2*epochs, 
#         verbose=1, shuffle=True, callbacks=[EarlyStopping(monitor='val_loss', mode='min', patience=6)])

Epoch 1/100
19/19 [==============================] - 10s 530ms/step - loss: 9.2050 - acc: 0.3657
Epoch 2/100
19/19 [==============================] - 6s 338ms/step - loss: 9.7487 - acc: 0.3964
Epoch 3/100
19/19 [==============================] - 6s 341ms/step - loss: 9.6959 - acc: 0.3957
Epoch 4/100
19/19 [==============================] - 6s 339ms/step - loss: 9.6509 - acc: 0.4001
Epoch 5/100
19/19 [==============================] - 6s 338ms/step - loss: 9.5613 - acc: 0.4078
Epoch 6/100
19/19 [==============================] - 7s 344ms/step - loss: 9.7887 - acc: 0.3937
Epoch 7/100
19/19 [==============================] - 6s 340ms/step - loss: 9.3049 - acc: 0.4236
Epoch 8/100
11/19 [================>.............] - ETA: 2s - loss: 10.1488 - acc: 0.3712

KeyboardInterrupt: ignored